In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns
import gzip
import shutil
import os.path
from scipy.io import mmread
import tifffile as tf
from scipy.spatial import ConvexHull, convex_hull_plot_2d
import json
from PIL import Image
import os
import anndata

In [ ]:
adata= anndata.read_h5ad('/mnt/DATA/ext_home/mariae/test_storage/maria_e/CRC/adata_with_factors.h5ad')

In [ ]:
#Selecting the cell populations to subcluster
interesting_clusters_2 = ['macrophage','CAF/monocyte','macrophage DNASE1L3+','macrophages','mast cell_1','monocyte','monocyte CCL4+ IL1B+','B cell active']

In [ ]:
subcluster_data_2 = adata[adata.obs['combined_clusters'].astype(str).isin(interesting_clusters_2)].copy()

In [ ]:
sc.pp.neighbors(subcluster_data_2, n_neighbors=7, n_pcs=0)
sc.tl.umap(subcluster_data_2,min_dist=0.1)
sc.tl.leiden(subcluster_data_2,resolution=1.0,key_added='myeloid_subclus_nei7_dist01_res1_bcellactive_included')

In [ ]:
sc.set_figure_params(scanpy=True, dpi=150,figsize=(10,10))
plt.rcParams['figure.facecolor'] = 'white'
sc.pl.umap(subcluster_data_2,color=['myeloid_subclus_nei7_dist01_res1_bcellactive_included','sample'],s=1,ncols=1,legend_loc='on data',save='myeloid_subclus_nei7_dist01_res1_bcellactive_included.pdf')

In [ ]:
#DEG
sc.tl.rank_genes_groups(subcluster_data_2,groupby='myeloid_subclus_nei7_dist01_res1_bcellactive_included')

In [ ]:
sc.pl.rank_genes_groups_dotplot(subcluster_data_2,groupby='myeloid_subclus_nei7_dist01_res1_bcellactive_included',n_genes=5,save='myeloid_subclus_nei7_dist01_res1_bcellactive_included_dotplot.pdf')

In [ ]:
#add previous annotations
sc.pl.umap(subcluster_data_2,color=['combined_clusters','sample'],s=1,ncols=1,legend_loc='on data',save='myeloid_subclustering_nei7_dist01_res1_bcellactive_originalannotationbeforesubclust.pdf')


In [ ]:
subcluster_data_2.write('/mnt/DATA/ext_home/mariae/test_storage/maria_e/CRC/CRC/xenium_processing/myeloid_subclustering/incluyendo_bcellactive/myeloid_subclustering_nei7_dist01_res2.h5ad')

In [ ]:
#get each clustering for each sample for tissuumaps
# Get unique sample tags
sample_tags = subcluster_data_2.obs['sample'].unique()

# Split the concatenated AnnData object into individual AnnData objects
individual_adatas = {}
for sample_tag in sample_tags:
    mask = (subcluster_data_2.obs['sample'] == sample_tag)
    individual_adatas[sample_tag] = subcluster_data_2[mask].copy()

In [ ]:
individual_adatas['0021489_Region_1'].write(("/mnt/DATA/ext_home/mariae/test_storage/maria_e/CRC/CRC/xenium_processing/myeloid_subclustering/incluyendo_bcellactive/0021489_Region_1"+'.h5ad'))

In [ ]:
individual_adatas['0021489_Region_2'].write(("/mnt/DATA/ext_home/mariae/test_storage/maria_e/CRC/CRC/xenium_processing/myeloid_subclustering/incluyendo_bcellactive/0021489_Region_2"+'.h5ad'))

In [ ]:
individual_adatas['0021492_Region_1'].write(("/mnt/DATA/ext_home/mariae/test_storage/maria_e/CRC/CRC/xenium_processing/myeloid_subclustering/incluyendo_bcellactive/0021492_Region_1"+'.h5ad'))

In [ ]:
individual_adatas['0021492_Region_2'].write(("/mnt/DATA/ext_home/mariae/test_storage/maria_e/CRC/CRC/xenium_processing/myeloid_subclustering/incluyendo_bcellactive/0021492_Region_2"+'.h5ad'))

In [ ]:
#annotate subclustering
adata= anndata.read_h5ad('/mnt/DATA/ext_home/mariae/test_storage/maria_e/CRC/CRC/xenium_processing/myeloid_subclustering/incluyendo_bcellactive/myeloid_subclustering_nei7_dist01_res2.h5ad')

In [ ]:
class_ann=pd.read_csv(r'/mnt/DATA/ext_home/mariae/test_storage/maria_e/CRC/CRC/xenium_processing/myeloid_subclustering/incluyendo_bcellactive/annotation_myeloid.csv',sep=';')

In [ ]:
class_ann

In [ ]:
adata

In [ ]:
cls=dict(zip(class_ann['Cluster'],class_ann['Interpretation']))
adata.obs['myeloid_subclus_cluster']=adata.obs['myeloid_subclus_nei7_dist01_res1_bcellactive_included'].astype(int).map(cls)

In [ ]:
adata

In [ ]:
adata.write('/mnt/DATA/ext_home/mariae/test_storage/maria_e/CRC/CRC/xenium_processing/myeloid_subclustering/incluyendo_bcellactive/myeloid_subclustering_annotated.h5ad')

In [ ]:
adata_full=anndata.read_h5ad('/mnt/DATA/ext_home/mariae/test_storage/maria_e/CRC/CRC/xenium_processing/myeloid_subclustering/incluyendo_bcellactive/adata_with_factors_viewer.h5ad')

In [ ]:
#add myeloid annotation from adata_sublustering to adata_main on the matching cells on both adatas.
adata_full.obs.loc[adata.obs.index, 'myeloid_subclus_cluster'] = adata.obs['myeloid_subclus_cluster']


In [ ]:
adata_full.write('/mnt/DATA/ext_home/mariae/test_storage/maria_e/CRC/CRC/xenium_processing/myeloid_subclustering/incluyendo_bcellactive/adata_myeloid_annotated.h5ad')

In [ ]:
adata_full